# DETR (End-to-End Object Detection with Transformers)  
- 논문 리뷰

## Abstract  
- 우리는 object detection을 direct set prediction problem으로 보는 방법을 제안한다  
- 우리의 접근법은 NMS(non maximum suppression)나 anchor generation과 같은 직접 디자인해야 하는 요소들을 제거함으로써 pipeline을 간소화시켰다  
- DETR의 주 구성요소는 bipartite matching을 통한 unique prediction을 하게 만드는 set-based global loss와 transformer encoder-decoder 구조다  
- 학습된 object queries의 고정된 small set이 주어지면 DETR은 object의 관계와 global image context를 판단하고 동시에 prediction의 final set을 반환한다  
- DETR은 최적화가 잘 된 Faster R-CNN과 비교해 COCO dataset에서 비슷한 성능을 보여준다  
- 또한 쉽게 일반화가 가능해 panoptic segmentation에도 적용할 수 있다

## Introduction  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DETR/figure/figure1.JPG?raw=true">


- object detection의 목표는 bounding box와 classification을 동시에 진행하는 것이다  
- 현재 detector들은 간접적으로 set prediction을 다루고 있다  
- 예를 들어, surroate regression, proposal을 통한 classification, anchor, window center 등이 있다 
- 이들의 성능은 거의 겹치는 prediction을 postprocessing하거나 anchor set을 디자인하거나 target box를 hueristic하게 anchor에 할당하는데 상당히 영향을 미친다  
- 이러한 pipeline을 간소화하기 위해 위와 같은 step들을 건너뛰어 direct set prediction 접근법을 제안한다  
- 우리는 object detection을 direct set prediction problem으로 생각해 training pipeline을 간소화한다  
- transformer 기반 encoder-decoder 구조를 채택한다  
- self attention mechanism이 중복 prediction을 제거하는 것처럼 set prediction의 제약을 쉽게 다룰 수 있도록 한다  
- DETR은 모든 object를 동시에 예측하고 predicted와 ground truth 사이의 bipartite matching을 수행하는 set loss function과 함께 end-to-end로 훈련된다  
- custom layer가 필요 없으며 CNN와 transformer를 가지고 간단하게 재구성할 수 있다  
- DETR의 주요 특징은 bipartite matching loss와 paralled decoding이다  
- DETR과 Faster R-CNN을 COCO dataset에서 성능을 비교한다  
- 그러나 DETR은 large object에 대해서만 좋은 성능을 보였다  
- 이는 transformer의 non-local computation 때문일 가능성이 높다  
- 즉, 작은 object에 대해서는 낮은 성능을 보여준다  
- 이는 FPN과 같은 방법을 적용한다면 해결될 것이라 기대한다  
- DETR은 long training schedule이 필요하며 auxiliary decoding loss도 사용해야 한다  
- DETR의 구조를 바꿔 panoptic segmentation도 수행할 수 있다  

## The DETR model  
- direct set prediction을 위해선 두 가지 구성 요소가 필요하다  
    - 1. predicted와 ground truth boxes 사이의 unique matching을 하게 만드는 set prediction loss  
    - 2. object의 set을 예측하고 이들의 관계를 모델링하는 architecture  
   
### Object detection set prediction loss  
- DETR은 N개의 고정된 사이즈의 prediction set을 decoder를 통해 추론한다  
- 이때 N은 이미지 안의 object의 수보다 더 많아야 한다  
- 학습 과정에서 주된 어려움 중 하나는 predicted와 이에 해당하는 ground truth의 score를 계산하는 것이다  
- 우리의 loss function은 predicted와 gt 사이에 최적화된 bipartite matching을 생성하고 object specific loss를 최적화 한다  
- ground truth를 $y$, set of prediction을 $\hat{y}=\left\{\hat{y}_i\right\}_{i=1}^N$이라 하자  
- 이때 N은 이미지에서 object의 갯수보다 크기 때문에 $y$는 $\emptyset$으로 패딩된 N 사이즈의 set이라 할 수 있다  
- 이러한 두 set 사이의 bipartite matching을 찾기 위해 우리는 가장 낮은 cost의 N개의 element $\sigma$에 대한 permutation을 탐색한다  
$$\hat{\sigma}=argmin_{\sigma\in\mathcal{G}_N}\sum_{i}^N\mathcal{L}_{match}\left(y_i,\, \hat{y}_{\sigma\left(i\right)}\right)$$  
- 이때 $\mathcal{L}_{match}\left(y_i, \hat{y}_{\sigma}\left(i\right)\right)$는 ground truth $y_i$와 index $\sigma\left(i\right)$의 prediction $\hat{y}$ 사이의 pair-wise matching cost이다  
- matching cost는 class prediction과 predicted gt의 유사성 두 가지를 고려한다  
- gt set의 각 element $i$는 $y_i=\left(c_i,b_i\right)$로 나타낼 수 있다  
- 이때 $c_i$는 target class label, $b_i\in\left[0,1\right]^4$는 gt box의 중심 좌표와 높이, 너비에 해당하는 좌표다  
- $\sigma\left(i\right)$의 prediction에 대해 클래스 $c_i$에 대한 확률을 $\hat{p}_{\sigma\left(i\right)}\left(ci\right)$, predicted box를 $\hat{b}_{\sigma\left(i\right)}$라 정의한다   
- matching을 찾는 과정은 modern detector인 proposal이나 anchor를 gt에 매칭하는 heuristic assignment rule과 같은 역할을 한다  
- 다른 점은 중복 없이 direct seg prediction을 위한 one-to-one matching을 한다는 것이다  
- 두 번째 step은 이전 step에서 매칭한 모든 pairs에 대해 Hungarian loss를 계산하는 것이다  
- 클래스 prediction과 box loss를 위해 negative log likelihood의 linear combination을 사용한다  
- 즉, cross entropy loss와 같은 의미를 가짐  
$$\mathcal{L}_{Hungarian}\left(y,\hat{y}\right)=\sum_{i=1}^N\left[-\log\hat{p}_{\hat{\sigma}_\left(i\right)}\left(c_i\right)+\mathcal{1}_{\left\{c_i\neq\emptyset\right\}}\mathcal{L}_{box}\left(b_i,\hat{b}_{\hat{\sigma}}\left(i\right)\right)\right]$$  
- 이때 $\hat{\sigma}$은 step 1에서 계산된 최적의 assignment이다  
- 실질적으로 우리는 class imbalance를 위해 $c_i=\emptyset$일 때 log-probability term을 factor 10 정도로 down-weight한다  
- 이는 Faster R-CNN의 negative/positive sample의 밸런스를 맞추는 작업과 비슷하다  
- object와 $\emptyset$ 사이의 cost는 prediction에 의존하지 않으며 이 경우에는 cost가 constant이다  
- 또한 log probabilities 대신 $\hat{p}_{\hat{\sigma}\left(i\right)}\left(c_i\right)$를 사용한다  
- 따라서 클래스가 정답인 경우에는 $\log\hat{p}_{\hat{\sigma}\left(i\right)}\left(c_i\right)$를 사용한다  

**Bounding box loss**  
- 식 2의 두 번째 항은 bounding box의 loss를 계산하는 부분이다  
- bounding box를 예측할 때는 large box와 small box의 오차가 비슷해도 스케일이 다르기 때문에 L1 loss와 GIoU(generalized IoU) loss의 linear combination으로 설정한다  
- 따라서 우리의 box loss는 $\mathcal{L}_{box}\left(b_i,\hat{b}_{\sigma\left(i\right)}\right)=\lambda_{iou}\mathcal{L}_{iou}\left(b_i,\hat{b}_{\sigma\left(i\right)}\right)+\lambda_{L1}\lVert b_i-\hat{b}_{\sigma\left(i\right)}\rVert_1$으로 정의한다    

### DETR architecture  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DETR/figure/figure2.JPG?raw=true">


- DETR의 architecture는 3가지로 이루어져 있다  
    - 1. compact feature representation을 추출하기 위한 CNN backbone  
    - 2. transformer encoder-decoder  
    - 3. final detection prediction을 하는 간단한 FFN  
    
**Backbone**  
- CNN backbone은 lower resolution feature map $f\in\mathcal{R}^{C\times H\times W}$를 생성한다  
- 일반적으로 $C=2048$, $H=\frac{H_0}{32}$, $W=\frac{W_0}{32}$이다  
- ResNet의 output convolution layer의 channel 수가 2048이므로

**Transformer encoder**  
- encoder는 input을 sequence로 받기 때문에 $\textbf{z}_0$의 spatial dimension을 1차원으로 낮춰야 한다  
- 즉, $d\times HW$가 된다  
- encoder는 multi head attention과 feed forward network로 이루어져 있다  
- transformer 구조는 순서와 무관하기 때문에 고정된 positional encoding을 추가했으며 이는 각 attention layer에 추가된다  
- 여기서 기존 ViT와의 차이점이 있는데 기존 방법은 encoder에 입력하기 전에 한 번만 positional encoding을 하지만 여기 논문에서는 모든 encoder layer에서 positional encoding을 한다  

**Transformer decoder**  
- decoder도 마찬가지로 standard transformer 구조를 따르며 multi head self attention, encoder-decoder attention mechanism을 사용한다  
- 이를 따라 사이즈 d의 N개 embedding을 형성한다  
- 이때 N은 이미지 내의 object 수보다 충분히 큰 수, d는 sequence의 길이다  
- decoder 역시 encoder와 마찬가지로 순서가 무관하기 때문에 input embedding에 positional encoding을 학습하며 이를 object queries라 한다  
- N object queries는 decoder에 의해 output embedding으로 변환된다  
- 그러면 FFN에 의해 output embedding은 box coordinates와 class label로 독립적으로 decoding된다  
- 모델은 self attention과 encoder-decoder를 사용하면서 모든 object에 대해 global하게 추론한다  
- 모델은 전체 이미지를 context로 사용할 수 있다  

**Prediction feed-forward networks (FFNs)**  
- final prediction은 ReLU activation layer, hidden dimension d layer, linear projection layer로 이루어진 3개의 perceptron에 연산된다  
- FFN은 normalized center coordinates, height, width를 예측하고 linear layer는 softmax를 적용해 class label을 예측한다  
- N개의 bounding box를 예측하는 것은 N을 고정했기 때문에 이미지 내에 존재하는 object의 갯수보다 많아야 하며 object가 존재하지 않는다는 것을 표현하기 위해 $\emptyset$을 사용한다  

**Auxiliary decoding losses**  
- auxiliary loss를 사용하는 것이 학습에 도움이 되었다  
- 각 decoder layer 앞에 prediction FFN과 Hungarian loss를 추가한다  
- 모든 prediction FFN은 파라미터를 공유한다  
- decoder layer에 있는 prediction FFN에 input을 normalize하기 위해 추가적인 shared layer norm을 사용한다  

## Experiments  
- 우리의 DETR은 COCO dataset에서 Faster R-CNN과 비교해 경쟁력 있는 성능을 보여준다  
- 또한 DETR의 구조를 활용해 panoptic segmentation task도 수행할 수 있다  

**Dataset**  
- COCO 2017 dataset으로 실험을 진행했으며 train은 118k, valid는 5k로 구성되어 있다  
- 이미지에는 평균적으로 7개의 instance, 최대 63개의 instance가 존재한다  

**Technical details**  
- DETR은 AdamW로 훈련하며 initial learning rate는 transformer에선 $10^{-4}$, backbone에선 $10^{-5}$, weight decay는 $10^{-4}$로 설정했다  
- 모든 transformer weight는 Xavier로 초기화하며 backbone은 ImageNet pre-trained ResNet과 Frozen Batch Normalization을 사용한다  
- Augmentation   
    - Resize 480~1333   
    - Random Crop $p=0.5$로 설정 후 800~1333으로 resize  
    - dropout rate는 default 0.1로 설정  
- inference step에서 몇몇 slot은 빈 클래스를 예측한다  
- 우리는 이 slot을 second highest scoring class로 대체해 2 AP의 성능을 끌어올렸다  

### Comparison with Faster R-CNN  
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DETR/figure/table1.JPG?raw=true">  

- DETR이 large instance에서는 Faster R-CNN보다 7.8% AP로 더 좋은 성능을 보여주지만 small instance에 대해서는 5.5% AP 안좋은 성능을 보여준다  

### Ablations  
- transformer decoder에서 attention mechanism은 feature representation을 파악하는 데 중요하다  
- 우리의 DETR의 architecture와 loss의 component들이 성능에 어떤 영향을 주는지 연구했다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DETR/figure/figure3.JPG?raw=true">  

**Number of encoder layers**  
- encoder layer 수를 바꿔가며 global image level self-attention의 중요성을 체크했다  
- encoder layer가 없으면 3.9%의 AP가 떨어지며 large object의 AP에 대해서는 6 AP가 떨어졌다  
- figure 3에서 encoder의 마지막 layer의 attention map을 시각화했다  
- encoder는 이미 instance들을 분리하는 것처럼 보이며 이는 decoder에서 object extraction과 localization을 단순화할 수 있다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DETR/figure/figure6.JPG?raw=true">  

**Number of decoder layers**  
- 각 decoder layer 이후에 auxiliary loss를 적용했다  
- 따라서 prediction FFN은 모든 decoder layer output의 object를 예측하도록 설계되어 있다  
- 우리는 각 decoder layer가 얼마나 중요한지 실험을 했다  
- set based loss로 인해 DETR은 NMS가 필요하지 않다  
- NMS는 first decoder layer의 성능을 개선시켜준다  
- 이는 single decoder layer가 object의 상호 상관관계를 계산할 수 없기 때문에 같은 object에 대해 multiple prediction을 하는 경향이 있다는 사실을 보여준다  
- 마지막 layer에서는 오히려 잘 예측한 box가 NMS를 통해 제거됨으로써 AP에 안좋은 영향이 미치는 것을 볼 수 있다  
- encoder의 attention map과 비슷하게 figure 6에서 decoder의 attention map을 시각화했다  
- instance마다 다른 색이 칠해진 것을 알 수 있다  
- 이는 decoder의 attention이 상당히 local하다는 것을 의미하고 대부분 머리나 다리 같은 사지에 attention을 한다는 것을 알 수 있다  

**Importance of FFN**  
- FFN의 중요성을 파악하기 위해 지우고 실험을 해봤다  
- 그 결과 학습 파라미터 수는 줄어들었지만 성능이 2.3 AP가 떨어졌다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DETR/figure/table3.JPG?raw=true"> 

**Importance of positional encodings**  
- 우리 모델에 2가지 positioinal encoder이 있는데 하나는 spatial positonal encoding, 다른 하나는 output positional encoding이다  
- 다양한 encoding 조합으로 실험을 했고 그 결과는 table 3에 나와 있다  
- 먼저 spatial encoding을 아예 삭제했을 때는 baseline에 비해 성능이 7.8 AP가 하락했다  
- 결과적으로 positional encoding을 사용하는 것이 성능 향상에 도움이 된다  


- 이 연구들을 통해 transformer에서 attention, FFN, encoder, decoder, positional encoding 모두 object detection을 수행하는 데 있어 성능 향상에 기여한다고 볼 수 있다  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DETR/figure/table4.JPG?raw=true"> 

**Loss ablations**  
- Loss function의 각 component들이 성능에 어떤 영향을 미치는지 확인했다  
- classification loss, L1 bounding box distance loss, GIoU loss 3가지로 구성되어 있으며 결과는 table 4에 있다  
- 결론적으로 모든 loss component들을 사용하는 것이 가장 성능이 좋았다  

### Analysis  
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DETR/figure/figure7.JPG?raw=true"> 

**Decoder output slot analysis**
- figure 7에서 서로 다른 slot에 대해 box prediction을 시각화했다  
- DETR은 각 query slot에 대해 다른 specialization을 학습한다  
- 특히 모든 slot에 box를 예측하는 모드가 있다 (빨간색 점으로 표시)  
- 우리는 이것이 COCO의 object 분포와 관련이 있다고 가정하고 있다   

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/DETR/figure/figure5.JPG?raw=true">

**Generalization to unseen numbers of instances**  
- COCO dataset에서 한 이미지에 같은 instance가 여러 개 있는 경우는 많지 않다  
- 그래서 DETR의 일반화 성능을 측정하기 위해 figure 5처럼 이미지 합성을 했다  
- 이 실험을 통해 각 object query에서 특정 클래스에만 좋은 성능을 내는 것이 아님을 확인했다  

## Conclusion  
- 우리는 direct set prediction을 위한 bipartite loss와 transformer 기반 object detection system을 제안했다  
- DETR은 COCO dataset에서 Faster R-CNN과 비교해 좋은 성능을 보여준다  
- 또한 flexible한 특징을 가지고 있어 panoptic segmentation task까지 수행할 수 있다  
- self attention 덕분에 큰 object에 대해선 Faster R-CNN보다 좋은 성능을 보여준다  
- 작은 object에 대해서도 detect를 하기 위해 노력을 할 필요가 있다  